In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn

import sys

sys.path.append('..')

from algorithms.random_policy import RandomPolicy
from algorithms.sequence_models.decision_sequence_policy import DTPolicy
from algorithms.sequence_models.decision_transformer.decision_transformer import DecisionTransformer
from algorithms.sequence_models.evaluate import evaluate_on_env
from data.door_key_dataset import DoorKeyDataset
from data.random_walk_dataset import RandomWalkDataset
from envs.door_key import DoorKeyEnv, DoorKeyEnvSmall
from data.trajectory import LimitedContextWrapper
from algorithms.sequence_models.config import TrainConfig
from algorithms.sequence_models.decision_transformer.trainer import TrainerDT
from envs.random_walk import RandomWalkEnv
from experiment import Experiment
import gymnasium as gym
from functools import partial

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device is ", device)

device is  cuda


In [2]:
from envs.carnival import get_carnival_env

/snap/pycharm-professional/387/plugins/python/helpers/pydev/pydevd_plugins/__init__.py:2: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('pydevd_plugins')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  __import__('pkg_resources').declare_namespace(__name__)
/snap/pycharm-professional/387/plugins/python/helpers/pydev/pydevd_plugins/extensions/__init__.py:2: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('pydevd_plugins.extensions')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  __import__('pkg_resources').declare_namespace(__name__)
/snap/pycharm-professional/387/plugins/python/helpers/pydev/pydevd_plugins/e

In [7]:
env = get_carnival_env()
obs, _ = env.reset()
obs.shape

(214, 160, 3)

In [30]:
import torch
import torch.nn as nn

class CNNFeatureExtractor(nn.Module):
    def __init__(self):
        super(CNNFeatureExtractor, self).__init__()
        # Define the convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        # Define the max pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Define the fully connected layers
        self.fc1 = nn.Linear(64 * 26 * 20, 512)  # Adjust the input size based on your image dimensions
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
    
    def forward(self, x):
        # Apply convolutional and pooling layers
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        # Flatten the output for the fully connected layers
        x = x.view(-1, 64 * 26 * 20)  # Adjust the size based on your image dimensions
        # Apply fully connected layers
        print(x.shape)
        # print(self.fc1(x).shape)
        # print('here')
        # 
        # x = torch.relu(self.fc1(x))
        # x = torch.relu(self.fc2(x))
        # x = torch.relu(self.fc3(x))
        # x = torch.relu(self.fc4(x))
        return x

# Create an instance of the CNN
cnn_model = CNNFeatureExtractor()
# Generate a random input tensor with the shape (batch_size, channels, height, width)
input_tensor = torch.randn(1, 3, 214, 160)  # Adjust the batch size and image dimensions as needed
# Forward pass through the CNN to get the feature vector
output_feature_vector = cnn_model(input_tensor)
print(output_feature_vector.shape)  # Print the shape of the output feature vector

torch.Size([1, 33280])
torch.Size([1, 33280])


In [31]:
cnn = CNNFeatureExtractor()
cnn(torch.from_numpy(obs).unsqueeze(0).float())
# [np.prod(p.shape) for p in cnn.parameters()]

RuntimeError: Given groups=1, weight of size [16, 3, 3, 3], expected input[1, 214, 160, 3] to have 3 channels, but got 214 channels instead

In [ ]:
# from minigrid.wrappers import ImgObsWrapper
# 
# inner_env = gym.make('MiniGrid-DoorKey-5x5-v0')
# inner_env_human = gym.make('MiniGrid-DoorKey-5x5-v0', render_mode='human')
# env = ImgObsWrapper(inner_env)
# env_human = ImgObsWrapper(inner_env_human)
# 
# 
# config = TrainConfig(max_eval_ep_len=env.max_steps, context_len=32,
#                      eval_model_interval=50)
# 
# # todo save the dataset for the experiment
# traj_dataset = DoorKeyDataset(env, n_trajectories=50_000, reward_scale=1)
# 
# dt_model = DecisionTransformer(
#     state_dim=traj_dataset.state_dim(),
#     act_dim=traj_dataset.action_dim(),
#     n_blocks=config.n_blocks,
#     h_dim=config.embed_dim,
#     context_len=config.context_len,
#     n_heads=config.n_heads,
#     drop_p=config.dropout_p,
# ).to(device)
# 
# make_dt_policy = partial(DTPolicy, model=dt_model, traj_dataset=traj_dataset, device=device, max_test_ep_len=config.max_eval_ep_len, context_length=config.context_len)
# 
# experiment = Experiment(
#     model_name='dt',
#     model=dt_model,
#     env_name='MiniGrid-DoorKey-5x5-v0',
#     env=env,
#     experiment_name='generalization',
#     traj_dataset=traj_dataset,
#     dataset_name=f'size={len(traj_dataset)}',
#     config=config,
#     device=device,
#     eval_policies_and_names=[
#         (make_dt_policy(rtg=rtg), f'dt,rtg={rtg}')
#         for rtg in np.linspace(0, 1.5, 10)
#     ]
# )